In [1]:
import pandas as pd
import time
import datetime

In [2]:
        import pickle
        with open('./csfail_ratios.pkl', 'rb') as f:
            result = pickle.load(f)
        print(result.loc[-30:])

         Round   Ratio
0    3920001.0    3.08
1    3920002.0  109.08
2    3920003.0    2.39
3    3920004.0    2.83
4    3920005.0    2.51
..         ...     ...
720  3920721.0    1.08
721  3920722.0    1.19
722  3920723.0    1.39
723  3920724.0    5.14
724  3920725.0    3.07

[725 rows x 2 columns]


In [37]:
print(result["Round"] == 3920001)


0      True
1      True
2      True
3      True
4      True
       ... 
121    True
122    True
123    True
124    True
125    True
Name: Round, Length: 126, dtype: bool


In [38]:

#list(map(lambda x: x.values()[3], l))